In [1]:
#Last time steps of long_run '2019-07-16T00:29:00.000000000'

In [2]:
import sys
import xarray as xr
import numpy as np
import os
import yaml
from random import randint
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ParcelsRandom, Variable

In [3]:
sys.path.append('/scratch/jvalenti/OParcels_runs/Source')
from OP_functions_graham import *

In [4]:
def simple_partition_function(coords, mpi_size=1):
    """A very simple partition function
    that assigns particles to processors
    """
    return np.linspace(0, mpi_size, coords.shape[0], endpoint=False, dtype=np.int32)

In [5]:
config = ['Parcels_outfall/outfall_test0.yaml']
local = 0

In [6]:
param = load_config(config)
#Definitions
start = datetime(param['startdate']['year'], param['startdate']['month'], param['startdate']['day']) #Start date
length = param['param']['length'] # Set Time length [days] 
dt = param['param']['dt'] #toggle between - or + to pick backwards or forwards 
N = param['param']['N'] # number of deploying locations
n = param['param']['n'] # 1000   # number of particles per location
dmin = param['param']['dmin'] #minimum depth
dd = param['param']['dd'] #max depth difference from dmin
name = param['file']['name'] #name output file
dtp = param['param']['dtp'] #how often particle released in hours
odt = param['param']['odt'] #how often data is recorded
rrr = param['param']['r'] #radious of particle deployment
MFc = param['param']['MFc']
paths = path(local)
clat = param['param']['lats']
clon = param['param']['lons']
duration = timedelta(days=length)
#Set deploy location
lon, lat, z = pandas_deploy(N,MFc,rrr,dd,dtp)
N = len(lat)

In [7]:
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name +'_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.zarr'
outfile = os.path.join('/scratch/jvalenti/OParcels_runs/Parcels_outfall/results/', fn)

In [8]:
####BUILD FIELDS FOR SIMULATION######
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables=filename_set(start,length,varlist,local)
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True, chunksize='auto')

#Find file names and variable names ###'Diat','Flag'###
varlist=['US','VS','WL','R','T','S','ssh','Bathy','Kz','totdepth','Vol']
filenames,variables=filename_set(start,length,varlist,local)

#Add Stokes Drift fields
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True, chunksize='auto')
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True, chunksize='auto')
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

#Add Vertical diffusivity coefficient field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw','time': 'time_counter'}
Kz = Field.from_netcdf(filenames['Kz'], variables['Kz'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(Kz)

#Add fields located at node T
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'deptht','time': 'time_counter'}
R = Field.from_netcdf(filenames['R'], variables['R'], dimensions,allow_time_extrapolation=True, chunksize='auto')
S = Field.from_netcdf(filenames['S'], variables['S'], dimensions,allow_time_extrapolation=True, chunksize='auto')
T = Field.from_netcdf(filenames['T'], variables['T'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(R)
field_set.add_field(S)
field_set.add_field(T)

#Add Bathymetry 2D field
dimensions = {'lon': 'glamt', 'lat': 'gphit'}
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True, chunksize='auto')
TD = Field.from_netcdf(filenames['totdepth'], variables['totdepth'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(Bth)
field_set.add_field(TD)

#Add Volume 3D field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw'}
Vol = Field.from_netcdf(filenames['Vol'], variables['Vol'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(Vol)


#Add SSH and Rivers 2D fields
dimensions = {'lon': 'glamt', 'lat': 'gphit','time': 'time_counter'}
SSH = Field.from_netcdf(filenames['ssh'], variables['ssh'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(SSH)

MPParticle = particle_maker(param)

/home/jvalenti/miniforge3/envs/Parcels_24/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/home/jvalenti/miniforge3/envs/Parcels_24/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(
/home/jvalenti/miniforge3/envs/Parcels_24/lib/python3.11/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mappin

In [9]:
pset = ParticleSet(field_set, MPParticle, lon=lon, lat=lat, depth=z, repeatdt = timedelta(hours=dtp))#,partition_function=simple_partition_function

In [11]:
KERNELS =  Advection + pset.Kernel(Buoyancy) + pset.Kernel(Stokes_drift) + pset.Kernel(turb_mix) + pset.Kernel(Displacement) + pset.Kernel(Unbeaching)+pset.Kernel(DeleteParticle)

pset.execute(KERNELS,
            runtime=duration, 
            dt=dt,
            output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=odt)))

INFO: Output files are stored in /scratch/jvalenti/OParcels_runs/Parcels_outfall/results/Outfalls_test_220190101_1n_20190103_1n.zarr.


KeyError: 'lon/.zarray'